<a id='top'></a>

[Jump to bottom](#bottom)

# Aspect Calculations
"Calculate aspect grid over the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid or we can't calculate covariates at
 the watershed scale.
 * **aspect_rch = calculate mean aspect for stream reach (zonal statistics)**
 * **aspect_cat = calculate mean aspect over catchments (zonal statistics)**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed."**



### Set environments and import modules

In [47]:
import arcpy
import os
import datetime
import time
import sys

arcpy.env.overwriteOutput = True
spref = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

20082021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\geomorphology
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


### Set variables and network data
Can run individual hucs one at a time by manully entering in "region" list ex regions = ['1090201'] or set up to iterate using predefined lists for Cook Inlet or Copper River.

In [54]:
#List of all Hucs in Cook Inlet
ci_huc_list = ['19020202', '19020301','19020302','19020302','19020401','19020402','19020501','19020503','19020503',
 '19020504','19020505','19020601','19020602','19020800']#list of all cook inlet hucs to be processed (if we want to iterate)
#List of all Hucs in Cook Inlet
cr_huc_list = ['19020101','19020102','19020103','19020104']#list of all copper river hucs to be processed
#Data folder for Cook Inlet
ci_data_folder = r"T:\\Aquatic\\AKSSF\\NHDPlus\\CookInlet_20201216"#folder containing NHDPLUS data
#Data folder for Copper River
cr_data_folder = r"T:\\Aquatic\\AKSSF\\NHDPlus\\CopperRiver_20210408"#folder containing NHDPLUS data

#Test Data Folder - hash out when running
data_folder = r"d:\\Basedata\\NHDPlus"#Test
roi = 'Test_' + str(time_stamp) + '_'

# Unhash below if iterating Cook Inlet
#roi = 'Cook_Inlet' + str(time_stamp) + "_"
#region = ci_huc_list
#data folder = ci_data_folder

# Unhash below if iterating Copper River
#roi = 'Copper_River_'+ str(time_stamp) + "_"
#region = cr_huc_list
#data folder = cr_data_folder

#Manually set region/s in region list - Hash out if using predifined list above
regions = ["19020401"] #8-digit HUC identifier for region to be processed (will be used as prefix for outputs)

identifierfield = "NHDPlusID" #Name of field storing Catchment/RCA field in input stream layer ex. NHDPlus = "NHDPLusID", rca streams = "reachid"
#Test network GDB
networkgdb = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\AKSSF\\hydrography\\AKSSF_Hydrography.gdb" #test output location

#Network GDB to store outputs
#networkgdb = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\Geomorphology"#network location to save final copies

Create temporary working directory and geodatabase. Set temp path = to directory on local machine used to run operations

In [67]:
temppath = r"D:\\GIS_temp"#Change to directory on local machine
dirname = 'AKSSF_Working'
tempgdbname = 'AKSSF_Working.gdb'
temp_dir = os.path.join(temppath,dirname)
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

Working Folder already created... D:\\GIS_temp\AKSSF_Working
Output location already exists D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb


Collect geodatabases in list that we can either iterate through or filter out using the region identifier


In [10]:
import arcpy
import os
nhd_gdbs=[]
arcpy.env.workspace = data_folder
gdbs = arcpy.ListWorkspaces("*", "FileGDB")
for gdb in gdbs:
    gdbpath = os.path.normpath(os.path.join(data_folder, gdb))
    nhd_gdbs.append(gdbpath)

nhd_gdbs

['d:\\Basedata\\NHDPlus\\NHDPLUS_H_19020401_HU8_GDB.gdb']

Collect watershed flowlines and catchments.
Must copy NHDPlusID to new text field because zonal statistics as table does not allow 'double' type fields to be used
for zone identification.

In [12]:
arcpy.env.overwriteOutput = True
catchments = []
flowlines = []
for region in regions:
    for gdb in nhd_gdbs:
        if region in gdb:
            arcpy.env.workspace = gdb
            datasets = arcpy.ListDatasets(feature_type='feature')
            for ds in datasets:
                for fc in arcpy.ListFeatureClasses(feature_dataset=ds):

                    if fc == "NHDPlusCatchment": # Grab Catchments
                        catchpath = os.path.join(arcpy.env.workspace, ds, fc)
                        catchid= 'NHDPlusCatchment_'+str(region)
                        catchcopy = arcpy.FeatureClassToFeatureClass_conversion(catchpath, outgdb, catchid)
                        arcpy.AddField_management( catchcopy, field_name='NHDPlusID_txt', field_type='Text')
                        arcpy.CalculateField_management( in_table=catchcopy, field = 'NHDPlusID_txt',
                                                         expression_type='PYTHON3', expression='!NHDPlusID!')
                        catchments.append(catchcopy)

                        print (catchcopy)
                        print ("")

                    if fc == "NHDFlowline": # Grab Flowlines
                        flowpath = os.path.join(arcpy.env.workspace, ds, fc)
                        flowid= 'NHDPlusFlowline_'+str( region)
                        flowcopy = arcpy.FeatureClassToFeatureClass_conversion( flowpath, outgdb, flowid)
                        arcpy.AddField_management( flowcopy, field_name='NHDPlusID_txt', field_type='Text')
                        arcpy.CalculateField_management(in_table=flowcopy,field = 'NHDPlusID_txt',
                                                        expression_type='PYTHON3', expression='!NHDPlusID!')
                        flowlines.append(flowcopy)

                        print (flowcopy)
                        print ("")

print(catchpath)
print ("")
print(flowcopy)
print ("")


D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\NHDPlusCatchment_19020401

D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\NHDPlusFlowline_19020401

d:\Basedata\NHDPlus\NHDPLUS_H_19020401_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment

D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\NHDPlusFlowline_19020401



Collect watershed elevation, stream and flow accumulation rasters.

In [38]:
arcpy.env.workspace = data_folder
elev_rasters = []
stream_rasters = []
flowacc_rasters = []
flowdir_rasters = []
raster_folders = arcpy.ListWorkspaces()
for region in regions:
    rasfol = "Rasters"+str(region)
    for folder in raster_folders:
        if rasfol in folder:
            arcpy.env.workspace = folder
            rasters = arcpy.ListRasters()
            for raster in rasters:
                if raster == 'elev_cm.tif': #Grab elevation raster
                    elev_raspath = os.path.normpath(os.path.join(folder, raster))
                    elev_rasters.append(elev_raspath)
                if raster == 'swnet.tif': #Get stream raster
                    stream_raspath = os.path.normpath(os.path.join(folder, raster))
                    stream_rasters.append(stream_raspath)
                if raster == 'fac.tif': #Get flow accumulation raster
                    flowacc_raspath = os.path.normpath(os.path.join(folder, raster))
                    flowacc_rasters.append(flowacc_raspath)
                if raster == 'fdr.tif': #Get flow direction raster
                    flowdir_raspath = os.path.normpath(os.path.join(folder, raster))
                    flowdir_rasters.append(flowdir_raspath)
print(raster_folders)
print(flowacc_rasters)
print(stream_rasters)
print(elev_rasters)
print(flowdir_rasters)


['d:\\\\Basedata\\\\NHDPlus\\HRNHDPlusRasters19020401', 'd:\\\\Basedata\\\\NHDPlus\\NHDPLUS_H_19020401_HU8_GDB.gdb', 'd:\\\\Basedata\\\\NHDPlus\\NHDPLUS_H_19020401_HU8_GDB.jpg', 'd:\\\\Basedata\\\\NHDPlus\\NHDPLUS_H_19020401_HU8_GDB.xml']
['d:\\Basedata\\NHDPlus\\HRNHDPlusRasters19020401\\fac.tif']
['d:\\Basedata\\NHDPlus\\HRNHDPlusRasters19020401\\swnet.tif']
['d:\\Basedata\\NHDPlus\\HRNHDPlusRasters19020401\\elev_cm.tif']
['d:\\Basedata\\NHDPlus\\HRNHDPlusRasters19020401\\fdr.tif']


Merge data together
UNK if we want to try merging all data together and then running all HUC8's together or run individually

In [ ]:
#Insert code to Merge datasets together for entire AKKSSF Region of Interest

Create Aspect raster

In [48]:
aspect_rasters = []
arcpy.env.snapRaster = elev_raspath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = elev_raspath
arcpy.env.workspace = temp_dir
from arcpy.sa import Aspect

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    print("")
    for elev_raster in elev_rasters:
        if region in elev_raster:
            # Start timing function
            iteration_start = time.time()
            asp_rasname = "Aspect_" + str(region)+'.tif'
            asp_rast = Aspect(in_raster= elev_raspath, method='Planar')
            asp_rast.save(os.path.join(temp_dir,asp_rasname))
            aspect_rasters.append(asp_rast)
            # End timing
            iteration_end = time.time()
            iteration_elapsed = int(iteration_end - iteration_start)
            iteration_success_time = datetime.datetime.now()
            # Report success
            print(f'Aspect raster for {region} completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")}'
                  f' (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
            print('----------')
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'All Aspect Rasters(s) derived at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

aspect_rasters


Aspect raster for 19020401 completed at 2021-08-20 10:11 (Elapsed time: 0:00:17)
----------
All Aspect Rasters(s) derived at 2021-08-20 10:11 (Elapsed time: 0:00:17)
----------


[D:\GIS_temp\AKSSF_Working\Aspect_19020401.tif]

Calculate mean catchment aspect

In [56]:
mn_cat_asp_tables = []
arcpy.env.workspace = outgdb

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    print("")
    # Start iter timing function
    iteration_start = time.time()
    for catchment in catchments:
        if region in catchment:
            for aspect_raster in aspect_rasters:
                if region in aspect_raster:
                    print(catchment)
                    print(aspect_raster)
    mn_cat_asp_name = "mn_cat_asp_" + str(region)
    mn_cataspzon_table = arcpy.ia.ZonalStatisticsAsTable(catchment, "NHDPlusID_txt", aspect_raster,mn_cat_asp_name,
                                                      "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")
    mn_cat_asp_tables.append(mn_cataspzon_table)
    # End iter timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f'Mean Aspect table for catchments in {region} completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")}'
          f' (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print('----------')
    print(mn_cataspzon_table)
    print(catchment)
    print(aspect_raster)
    print("----------")

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'Mean aspect for all catchments derived at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

#Merge all catchment aspect tables together and save to network gdb
arcpy.env.workspace = networkgdb
cat_mn_asp_table_name = roi + "cat_mn_asp"
cat_mn_asp = arcpy.management.Merge(mn_cat_asp_tables, cat_mn_asp_table_name)
print(cat_mn_asp)


Mean Aspect table for catchments in 19020401 completed at 2021-08-20 10:42 (Elapsed time: 0:00:23)
----------
D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\mn_cat_asp_19020401
D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\NHDPlusCatchment_19020401
D:\GIS_temp\AKSSF_Working\Aspect_19020401.tif
----------
Mean aspect for all catchments derived at 2021-08-20 10:42 (Elapsed time: 0:00:23)
----------


<Result 'C:\\\\Users\\\\dwmerrigan\\\\Documents\\\\GitHub\\AKSSF\\\\hydrography\\\\AKSSF_Hydrography.gdb\\Test_20082021_cat_mn_asp'>

Calculate mean stream reach aspect


In [57]:
mn_flow_asp_tables = []
arcpy.env.overwriteOutput = True
arcpy.env.workspace = outgdb

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    # Start iter timing function
    iteration_start = time.time()
    for flowline in flowlines:
        if region in flowline:
            for aspect_raster in aspect_rasters:
                if region in aspect_raster:
                    print(flowline)
                    print(aspect_raster)
    mn_flow_asp_name = "mn_flow_asp_" + str(region)
    mn_flowaspzon_table = arcpy.ia.ZonalStatisticsAsTable(flowline, "NHDPlusID_txt", aspect_raster,mn_flow_asp_name,
                                                      "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")
    mn_flow_asp_tables.append(mn_flowaspzon_table)
    # End iter timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f'Mean Aspect table for stream reaches in {region} completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")}'
          f' (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
    print(mn_flowaspzon_table)
    print(flowline)
    print(aspect_raster)
    print('----------')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'Mean aspect for all stream reaches derived at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

#Merge all stream aspect tables together and save to network gdb
arcpy.env.workspace = networkgdb
str_mn_asp_table_name = roi + "str_mn_asp"
str_mn_asp = arcpy.management.Merge(mn_flow_asp_tables, str_mn_asp_table_name)
print(str_mn_asp)

Mean Aspect table for stream reaches in 19020401 completed at 2021-08-20 10:43 (Elapsed time: 0:00:06)
D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\mn_flow_asp_19020401
D:\\GIS_temp\AKSSF_Working\AKSSF_Working.gdb\NHDPlusFlowline_19020401
D:\GIS_temp\AKSSF_Working\Aspect_19020401.tif
----------
Mean aspect for all stream reaches derived at 2021-08-20 10:43 (Elapsed time: 0:00:06)
----------
C:\\Users\\dwmerrigan\\Documents\\GitHub\AKSSF\\hydrography\\AKSSF_Hydrography.gdb\Test_20082021_str_mn_asp


Calculate % North weighted

In [68]:
north_rasters = []
north_wt_fdr_rasters = []
per_nor_rasters = []

arcpy.env.workspace = temp_dir

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    # Start iter timing function
    iteration_start = time.time()
    for aspect_raster in aspect_rasters:
        if region in aspect_raster:
            norast_name='north_asp_' + str(region) + ".tif"
            norast_path = os.path.join(temp_dir, norast_name)
            asp_rast = aspect_raster
            per_nor_rast = arcpy.ia.RasterCalculator('Con((%s>=0)&(asp_rast<=45),1,Con((asp_rast<=360)&(asp_rast>=315),1,0))'); per_nor_rast.save(norast_path)

            north_rasters.append(per_nor_rast)
            # for flowdir_raster in flowdir_rasters:
            #     if region in flowdir_raster:
            #         nor_wt_rast_name = 'norwt_fdr_'+str(region)
            #         nor_wt_fdr = arcpy.FlowAccumulation_ra(flowdir_raster, nor_wt_rast_name, nrast,
            #                                                dataType = "INTEGER")
            #         nor_wt_fdr.save(nor_wt_rast_name)
            #         north_wt_fdr_rasters.append(nor_wt_fdr)
            #         for flowacc_raster in flowacc_rasters:
            #             if region in flowacc_raster:
            #                 per_nor_ras_name = 'per_nor_fac_' + str(region)
            #                 per_nor_ras = Raster(nor_wt_fdr) / Raster(flowacc_raster) * 100
            #                 per_nor_ras.save(per_nor_ras_name)
            #                 per_nor_rasters.append(per_nor_ras)
    # End iter timing
    iteration_end = time.time()
    iteration_elapsed = int(iteration_end - iteration_start)
    iteration_success_time = datetime.datetime.now()
    # Report success
    print(f' Percent North Aspect for {region} completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")}'
          f' (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'Percent North Aspect for all regions completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

print(north_rasters)
print(north_wt_fdr_rasters)
print(per_nor_rasters)

 Percent North Aspect for 19020401 completed at 2021-08-20 11:56 (Elapsed time: 0:00:20)
Percent North Aspect for all regions completed at 2021-08-20 11:56 (Elapsed time: 0:00:20)
----------
[]
[]
[]


<a id='bottom'></a>